In [1]:
## A singular interface to run icp on a scan
## Goal is to get the weld center predictions
## Function should specify 
#  - the version of ICP and the ICP args
#  - the initial transfrom



In [2]:
# Importing (from) libs
import torch
import numpy as np
import sklearn
import os, errno
import sys
from datetime import datetime
import time
import csv
import open3d as o3d
import copy

import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from math import floor, ceil
from scipy.stats import mode

from Instseg_model import MultiLayerFastLocalGraphModelV2 as model1
from dataset import pcloader
from graph_generation import gen_multi_level_local_graph_v3

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Importing from ExploringDBSCAN
from exploringDBSCAN import configure_model, \
                            get_weld_instances_from_3D_clustering, \
                            get_matching_template, \
                            compute_FPFH_for_weld_points_2D, \
                            compute_FPFH, global_reg_by_RANSAC, \
                            global_reg_by_FGR, global_reg_by_mean, \
                            global_reg_by_median, \
                            global_reg_by_mid, \
                            transform_to_xyztheta, \
                            point_to_point_ICP, \
                            point_to_plane_ICP, \
                            gen_weld_templates_by_params, \
                            gen_volumetric_weld_templates_by_params

In [4]:
## Setup: The model, templates, scan_path

# model
model_params_path = "./_model/train1-fix3/2023_06_22_17_01_13/params_epoch497_for_min_test_loss.pt"
model = configure_model(model_params_path, verbose=True)

# templates
vol_templates_dict = gen_volumetric_weld_templates_by_params()
# templates_dict = gen_weld_templates_by_params()
# 2circle_2D_template_dict
# 2circle_vol_template_dict


Model Setup Time (secs) : 0.3090033531188965


In [5]:
scan_path = "./_data/scans/LH-1--2312014-NoResult-l-2023_06_02-7-56-18-48.ply"

In [8]:
def runICPonScan(scan_path, model, icp_templates, \
                 icp_init_mode, icp_type, \
                 icp_max_correspondence_difference, voxel_size, \
                 dir_path):
    """
    icp_init_mode: "ransac", "fgr", "mid", "mean", or "median"
    icp_type : "point_to_point" or "point_to_plane"
    """
    
    eps = 2
    min_samples = 20
    inst_weight = 1
    tag = "XYZ_"
    voxel_size = 0.2
    
    # Convergence Criteria per ICP type
    pt2pt_criteria = o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.0000000001,
                                       relative_rmse=0.0000000000000000001,
                                       max_iteration=5000)
    
    pt2pl_criteria = o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.0000000001,
                                       relative_rmse=0.0000000000000000001,
                                       max_iteration=5000)
    
    ## ICP Type Objects
    pt2pt_estimation = o3d.t.pipelines.registration.TransformationEstimationPointToPoint()
    pt2pl_estimation = o3d.t.pipelines.registration.TransformationEstimationPointToPlane()
    
    ## Example callback_after_iteration lambda function:
    callback_after_iteration = lambda updated_result_dict : print("Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
    updated_result_dict["iteration_index"].item(),
    updated_result_dict["fitness"].item(),
    updated_result_dict["inlier_rmse"].item()))
    
    ### welds: a list of dicts representing each welds encapsulating respective min/max/mean/median/modes/etc
    ### scan_data --> removing too small and too large clusters --> x|y|z|cls_preds|inst_preds|clustering|cls_clean|inst_clean
    welds, scan_data = get_weld_instances_from_3D_clustering(model, scan_path, eps, min_samples, inst_weight, tag=tag, dir_path=dir_path+"clustering/")
    
    weld_PointClouds = []
    template_PointClouds = []
    template_ICPTransforms = []
    for idx, weld in enumerate(welds):


        ## 3. Get equivalent template for weld type
        template = get_matching_template(weld, icp_templates)


        ## 3.5 Compute FPFH Features --> needed for fast global registration
        ## Added to weld dict
        ## For weld
        compute_FPFH_for_weld_points_2D(weld, voxel_size)
        ## For Template
        if "down" not in template.keys() or "fpfh" not in template.keys():
            template_down, template_fpfh = compute_FPFH(template["xyz"], voxel_size)
            template["down"], template["fpfh"] = template_down, template_fpfh ## [possible RED FLAG] -- verify the templates dict is updated globally and not recomputed al the tiem
            print("Added downsampling and fpfh entries to template dict")
            
        ## Constructing Point Cloud Objects for weld 
        weld_pcd = o3d.t.geometry.PointCloud()
        weld_pcd.point.positions = o3d.core.Tensor(weld["xyz"])

        template_pcd = o3d.t.geometry.PointCloud()
        template_pcd.point.positions = o3d.core.Tensor(template["xyz"])
        
            
        ## Initial Transformation: Loose Registration
        T_init = None
        if icp_init_mode == 'ransac':
            T_init, _, _, _ = global_reg_by_RANSAC(template, weld, voxel_size)
        
        elif icp_init_mode == 'fgr':
            T_init, _, _, _ = global_reg_by_FGR(template, weld, voxel_size)

        elif icp_init_mode == 'mid':
            T_init, _, _, _ = global_reg_by_mid(template, weld)

        elif icp_init_mode == 'mean':
            T_init, _, _, _ = global_reg_by_mean(template, weld)
            
        elif icp_init_mode == 'median':
            T_init, _, _, _ = global_reg_by_median(template, weld)
        
        T_init = o3d.core.Tensor(T_init)
            
            
        ## ICP Registration: Tighter Registration
        registation_icp = None
        icp_time = None

        if icp_type == "point_to_point":
            s = time.time()
            registration_icp =  o3d.t.pipelines.registration.icp(template_pcd, weld_pcd, max_correspondence_distance,
                                        T_init, pt2pt_estimation, pt2pt_criteria, voxel_size,
                                        callback_after_iteration=callback_after_iteration)
            icp_time = time.time() - s
            
        elif icp_type == "point_to_plane":
            s = time.time()

            registration_icp =  o3d.t.pipelines.registration.icp(template_pcd, weld_pcd, max_correspondence_distance,
                                        T_init, pt2pl_estimation, pt2pl_criteria, voxel_size,
                                        callback_after_iteration=callback_after_iteration)
            icp_time = time.time() - s

        print(f"Time taken by ICP ({weld['tag']}): ", icp_time)
        print("Inlier Fitness: ", registration_icp.fitness)
        print("Inlier RMSE: ", registration_icp.inlier_rmse)
        print()
       
    
        ## collecting source and target point clouds and the transformation for inspection/visualization
        weld_PointClouds.append(copy.deepcopy(weld_pcd))
        template_PointClouds.append(template_pcd)
        template_ICPTransforms.append(registration_icp.transformation)
        
        return weld_PointClouds, template_PointClouds, template_ICPTransforms
        


In [9]:
weld_PCs, template_PCs, template_Ts = runICPonScan(scan_path, model, vol_templates_dict, \
                                                   "mid", "point_to_point", \
                                                   10, 1, \
                                                   "./function_check2/")

Scan Inference Time (secs):  15.580561637878418



NameError: name 'template_xyz' is not defined

<Figure size 640x480 with 0 Axes>